In [2]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_openai import  ChatOpenAI
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from dotenv import load_dotenv
from typing import List

load_dotenv()

True

In [3]:
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="Person's name")
    age: int = Field(description="Person's age")

class Information(BaseModel):
    """Information to extract"""
    people: List[Person] = Field(description="List of info about people")

In [5]:
extraction_function = convert_pydantic_to_openai_function(Information)

In [21]:
model = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided, do not guess, explicitly provided the answers"),
    ("human", "{input}"),
])

extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [22]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [24]:
extraction_chain.invoke("Hijab is 21 years & his mother name is Rukhsana.")

[{'name': 'Hijab', 'age': 21}]